In [1]:
from functions import *
import pandas as pd
import seaborn as sns
import statistics
from matplotlib import pyplot as plt
%matplotlib inline

blocks = pd.read_csv("./raw_data/pr/data_block.csv", names=["BLOCK", "NS"])
ecdsa_checks = pd.read_csv("./raw_data/pr/data_ecdsa.csv", names=["BLOCK", "NS"])
shielded_spend = pd.read_csv("./raw_data/pr/data_shieldedSpend.csv", names=["BLOCK", "NS"])
shielded_output = pd.read_csv("./raw_data/pr/data_shieldedOutput.csv", names=["BLOCK", "NS"])

mall_hash = pd.read_csv("./raw_data/pr/data_malleability_hash.csv", names=["BLOCK", "NS"])
mall_bindingSig = pd.read_csv("./raw_data/pr/data_malleability_bindingSig.csv", names=["BLOCK", "NS"])

In [2]:
#ecdsa_checks.tail()

In [3]:
block_inputs = group_by_block(ecdsa_checks)
block_spends = group_by_block(shielded_spend)
block_outputs = group_by_block(shielded_output)
block_mall = group_by_block(mall_hash)
block_bindingSig = group_by_block(mall_bindingSig)

In [4]:
block_inputs.loc['000000000074009a7fae00e0b3350ff3c05ff2c10597398578e361f8bd12b3d4']

[264645,
 221724,
 200810,
 179219,
 15428,
 12859,
 14088,
 12514,
 175786,
 9470,
 268826,
 133641,
 174014,
 114146,
 109969,
 111188,
 112379,
 110303,
 110475,
 111356,
 112241,
 149577,
 113822,
 111362,
 109865,
 111530,
 110692,
 111019,
 111137,
 111852,
 149972,
 111471,
 109738,
 111529,
 8068,
 6529,
 6556,
 6208,
 6287,
 6202,
 6254,
 6256,
 6263,
 6207,
 6339,
 6228,
 6223,
 6291,
 6179,
 6245,
 6223,
 6211,
 6169,
 6287,
 6221,
 6203,
 6240,
 6187,
 893,
 293,
 161,
 259,
 160,
 403,
 196,
 210,
 194,
 171,
 190,
 202,
 178,
 175,
 183,
 187,
 183,
 165,
 197,
 203,
 179,
 171,
 172,
 194,
 152,
 183,
 166,
 163,
 172]

In [5]:
avg_input = median_percent_of_block(block_inputs, blocks)
avg_spend = median_percent_of_block(block_spends, blocks)
avg_output = median_percent_of_block(block_outputs, blocks)
avg_mall = median_percent_of_block(block_mall, blocks)
avg_bindingSig = median_percent_of_block(block_bindingSig, blocks)

In [6]:
avg_input.describe()


,NS
count,162631.000000
mean,0.003021
std,0.079481
min,0.000001
25%,0.000345
50%,0.000975
75%,0.003040
max,23.551398


In [7]:
avg_spend.describe()

,NS
count,1.528390e+05
mean,2.261455e+01
std,4.805002e+01
min,2.160838e-07
25%,4.426981e-04
50%,1.358111e+01
75%,3.357414e+01
max,1.126036e+04


In [8]:
avg_output.describe()

,NS
count,152839.000000
mean,32.284884
std,84.369084
min,0.000003
25%,8.071293
50%,22.468245
75%,48.270758
max,8744.125705


In [9]:
avg_mall.describe()

,NS
count,169801.000000
mean,0.206009
std,6.951162
min,0.000117
25%,0.069300
50%,0.120371
75%,0.212235
max,2840.057648


In [10]:
avg_bindingSig.describe()

,NS
count,152749.000000
mean,2.070343
std,3.099729
min,0.002175
25%,0.800487
50%,1.447476
75%,2.739609
max,306.091120


In [11]:
total_input = total_percent_of_block([block_inputs], blocks)
total_spend = total_percent_of_block([block_spends], blocks)
total_output = total_percent_of_block([block_outputs], blocks)
total_mall_sapling = total_percent_of_block([block_mall, block_bindingSig], blocks)
total = total_percent_of_block([block_bindingSig, block_mall, block_inputs, block_spends, block_outputs], blocks)

In [12]:
total_input.describe()

,NS
count,427177.000000
mean,0.026385
std,2.077242
min,0.000000
25%,0.000000
50%,0.000000
75%,0.010061
max,770.116993


In [13]:
total_spend.describe()

,NS
count,427177.000000
mean,10.422165
std,32.207859
min,0.000000
25%,0.000000
50%,0.000000
75%,5.776467
max,11260.364276


In [14]:
total_output.describe()

,NS
count,427177.000000
mean,14.707641
std,54.185102
min,0.000000
25%,0.000000
50%,0.000000
75%,20.904370
max,8744.125705


In [15]:
total_mall_sapling.describe()

,NS
count,427177.000000
mean,1.042407
std,4.990035
min,0.000000
25%,0.000000
50%,0.000000
75%,1.689602
max,2840.057648


In [16]:
total.describe()

,NS
count,427177.000000
mean,26.198597
std,66.697452
min,0.000000
25%,0.000000
50%,0.020361
75%,58.127806
max,11478.142186
